In [1]:
!pip install --upgrade pip
!pip install pyarrow
!pip install pandas
!pip install duckdb
!pip install hnswlib
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa
import duckdb
import hnswlib

In [2]:
context_data_recorder_pq_read = pq.read_table('context_data_recorder.parquet')
context_data_recorder_pq_read_df = context_data_recorder_pq_read.to_pandas()
print("context_data_recorder_pq_read_df", context_data_recorder_pq_read_df.head())
print(context_data_recorder_pq_read_df.iloc[:1]["label"].values[0])

context_data_recorder_pq_read_df                                       embedding_data  \
0  [-0.24175236, -0.17489734, 0.27527738, -0.3626...   
1  [0.1372665, -0.20390767, 0.3732293, -0.2372347...   
2  [-0.106324054, -0.20140041, -0.019513553, -0.2...   
3  [0.20723782, -0.2109229, 0.22755651, -0.188208...   
4  [-0.060116727, -0.10828238, 0.2030715, -0.1245...   

                                        resource_uri  \
0  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
1  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
2  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
3  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
4  /Users/jeff/src/chroma/examples/yolov3_embeddi...   

                                  metadata_list  \
0        {'location': 'Chicago', 'quality': 60}   
1   {'location': 'San Francisco', 'quality': 0}   
2  {'location': 'San Francisco', 'quality': 38}   
3        {'location': 'Atlanta', 'quality': 86}   
4        {'location': 'Atlanta', 'qual

In [3]:
object_data_recorder_pq_read = pq.read_table('object_data_recorder.parquet')
object_data_recorder_pq_read_df = object_data_recorder_pq_read.to_pandas()
print("object_data_recorder_pq_read_df", object_data_recorder_pq_read_df.head())
print(object_data_recorder_pq_read_df.iloc[:1]["infer"].values[0])

object_data_recorder_pq_read_df                                       embedding_data  \
0  [-0.031979177, -1.2179449, 0.41527477, -0.0050...   
1  [0.38920435, -0.033903435, -0.030004164, 0.477...   
2  [0.29277077, 1.0449085, -0.6858237, 0.23218007...   
3  [0.012765436, 0.31523734, -0.27020937, 0.51416...   
4  [-0.95235604, -1.5869737, -0.029858245, -0.269...   

                                        resource_uri  \
0  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
1  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
2  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
3  /Users/jeff/src/chroma/examples/yolov3_embeddi...   
4  /Users/jeff/src/chroma/examples/yolov3_embeddi...   

                                       metadata_list  \
0  {'iou': 0.9026501178741455, 'score': 0.9997622...   
1          {'iou': 0.0, 'score': 0.8971673846244812}   
2  {'iou': 0.642760157585144, 'score': 0.88095790...   
3  {'iou': 0.6039116978645325, 'score': 0.8784567...   
4  {'iou': 0.8

In [4]:
# print(duckdb.query('''
# SELECT COUNT(*)
# FROM 'test_data_to_record.parquet'
# WHERE metadata.location ILIKE '%San Francisco%';
# ''').fetchall())
print("Record in test")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'context_data_recorder.parquet'
;
''').fetchall())
print("Record in train")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'object_data_recorder.parquet'
;
''').fetchall())

Record in test
[(50,)]
Record in train
[(254,)]


In [5]:
import hnswlib
import numpy as np

#dim = 16
# num_elements = 10000

# Generating sample data
# data = np.float32(np.random.random((num_elements, dim)))

# We split the data in two batches:
data1 = context_data_recorder_pq_read_df['embedding_data'].to_numpy().tolist()
dim = len(data1[0])
num_elements = len(data1) 
# print("dimensionality is:", dim)
# print("total number of elements is:", num_elements)
# print("max elements", num_elements//2)

concatted_data = data1 
# print("concatted_data", len(concatted_data))

# Declaring index
p = hnswlib.Index(space='l2', dim=dim)  # possible options are l2, cosine or ip

# Initing index
# max_elements - the maximum number of elements (capacity). Will throw an exception if exceeded
# during insertion of an element.
# The capacity can be increased by saving/loading the index, see below.
#
# ef_construction - controls index search speed/build speed tradeoff
#
# M - is tightly connected with internal dimensionality of the data. Strongly affects the memory consumption (~M)
# Higher M leads to higher accuracy/run_time at fixed ef/efConstruction

p.init_index(max_elements=len(data1), ef_construction=100, M=16)

# Controlling the recall by setting ef:
# higher ef leads to better accuracy, but slower search
p.set_ef(10)

# Set number of threads used during batch search/construction
# By default using all available cores
p.set_num_threads(4)

# print(data1[0])

# print("Adding first batch of %d elements" % (len(data1)))
p.add_items(data1)

# Query the elements for themselves and measure recall:
labels, distances = p.knn_query(data1, k=1)
print(labels)
print(distances)
print("Recall for the first batch:", np.mean(labels.reshape(-1) == np.arange(len(data1))), "\n")

[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [23]
 [24]
 [25]
 [26]
 [27]
 [28]
 [29]
 [30]
 [31]
 [32]
 [33]
 [34]
 [35]
 [36]
 [37]
 [38]
 [39]
 [40]
 [41]
 [42]
 [43]
 [44]
 [45]
 [46]
 [47]
 [48]
 [49]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Recall for the first batch: 1.0 



In [6]:
def unpack_annotations(embeddings):
    # Get and unpack inference data
    # for embedding in embeddings:
    #     print(embedding)

    annotations = [
        embedding['infer']["annotations"]
        for embedding in embeddings
    ]  # Unpack JSON
    # print("annotations", annotations)
    annotations = [
        annotation for annotation_list in annotations for annotation in annotation_list
    ]  # Flatten the list
    # print("annotations", annotations)

    # categories_by_uid = {
    #     annotation["id"]: annotation["category_id"] for annotation in annotations
    # }
    # print(categories_by_uid)

    # # Unpack embedding data
    embeddings = [embedding["embedding_data"] for embedding in embeddings]

    embedding_vectors_by_category = {}
    for embedding_annotation_pair in zip(embeddings, annotations):
        data = np.array(embedding_annotation_pair[0])
        print("embedding_annotation_pair[1]", embedding_annotation_pair)
        category = embedding_annotation_pair[1]['category_id'] #categories_by_uid[embedding["target"]]
        if category in embedding_vectors_by_category.keys():
            embedding_vectors_by_category[category] = np.append(
                embedding_vectors_by_category[category], data[np.newaxis, :], axis=0
            )
        else:
            embedding_vectors_by_category[category] = data[np.newaxis, :]

    return embedding_vectors_by_category

# Get the training embeddings. This is the set of embeddings belonging to datapoints of the training dataset, and to the first created embedding set.
context_embedding_vectors_by_category = unpack_annotations(context_data_recorder_pq_read_df.to_dict('records'))
print("context_embedding_vectors_by_category", context_embedding_vectors_by_category)
# print("context_data_recorder_pq_read_df.to_dict('records')", context_data_recorder_pq_read_df.head())


embedding_annotation_pair[1] (array([-0.24175236, -0.17489734,  0.27527738, ...,  0.19632292,
        0.09433545,  0.80175096], dtype=float32), {'bbox': array([161.40570068, 408.10144043, 144.48736572,  68.88256836]), 'category_id': 56, 'category_name': 'chair', 'id': 'fce4d8a6-7e89-4956-8aca-e16643638837', 'label_id': 'c99278a3-6efc-4efe-835c-cb3b6d04a46b', 'metadata': {'iou': 0.9026501178741455, 'score': 0.999762237071991}})
embedding_annotation_pair[1] (array([ 0.1372665 , -0.20390767,  0.3732293 , ..., -0.18009104,
       -0.04469211,  0.608395  ], dtype=float32), {'bbox': array([147.56387329, 193.01312256,  14.92988586,  32.22669983]), 'category_id': 40, 'category_name': 'wine glass', 'id': '943a77c2-3593-4e03-a192-7b797eea8b0c', 'label_id': '', 'metadata': {'iou': 0.0, 'score': 0.8971673846244812}})
embedding_annotation_pair[1] (array([-0.10632405, -0.20140041, -0.01951355, ...,  0.20097737,
        0.17402977,  0.3637258 ], dtype=float32), {'bbox': array([384.71234131, 187.26092

In [11]:
inv_covs_by_category = {}
means_by_category = {}
for category, embeddings in context_embedding_vectors_by_category.items():
    # print(f"Computing mean and covariance for label category {category}")

    # Compute the mean and inverse covariance for computing MHB distance
    # print(f"category: {category} samples: {embeddings.shape[0]}")
    if embeddings.shape[0] < (embeddings.shape[1] + 1):
        # print(f"not enough samples for stable covariance in category {category}")
        continue
    cov = np.cov(embeddings.transpose())
    try:
        inv_cov = np.linalg.inv(cov)
    except np.linalg.LinAlgError as err:
        # print(f"covariance for category {category} is singular")
        continue
    mean = np.mean(embeddings, axis=0)
    inv_covs_by_category[category] = inv_cov
    means_by_category[category] = mean

# print(means_by_category[5])
target_datapoints = context_data_recorder_pq_read_df.to_dict('records') #+ panda_train_table.to_dict('records')
# print(target_datapoints)

# Process each datapoint's inferences individually. This is going to be very slow.
# This is because there is no way to grab the corresponding metadata off the datapoint.
# We could instead put it on the embedding directly ?
inference_metadata = {}
for idx, datapoint in enumerate(target_datapoints):
    inferences = datapoint['infer']["annotations"]
    embeddings = [datapoint["embedding_data"]]
    print("embeddings", len(embeddings), embeddings)
    print("inferences", len(inferences), inferences)

    for i in range(len(inferences)):
        # print("i", i, "inferences[i]", inferences[i])
        emb_data = embeddings[i]
        category = inferences[i]["category_id"]
        if not category in inv_covs_by_category.keys():
            continue
        mean = means_by_category[category]
        inv_cov = inv_covs_by_category[category]
        delta = np.array(emb_data) - mean
        squared_mhb = np.sum((delta * np.matmul(inv_cov, delta)), axis=0)
        if squared_mhb < 0:
            print(f"squared distance for category {category} is negative")
            continue
        distance = np.sqrt(squared_mhb)
        # print(datapoint)
        inference_metadata[datapoint["resource_uri"]] = distance
        print("distance", distance)

# dict to json
import json


embeddings 1 [array([-0.24175236, -0.17489734,  0.27527738, ...,  0.19632292,
        0.09433545,  0.80175096], dtype=float32)]
inferences 17 [{'bbox': array([161.40570068, 408.10144043, 144.48736572,  68.88256836]), 'category_id': 56, 'category_name': 'chair', 'id': 'fce4d8a6-7e89-4956-8aca-e16643638837', 'label_id': 'c99278a3-6efc-4efe-835c-cb3b6d04a46b', 'metadata': {'iou': 0.9026501178741455, 'score': 0.999762237071991}}
 {'bbox': array([147.56387329, 193.01312256,  14.92988586,  32.22669983]), 'category_id': 40, 'category_name': 'wine glass', 'id': '943a77c2-3593-4e03-a192-7b797eea8b0c', 'label_id': '', 'metadata': {'iou': 0.0, 'score': 0.8971673846244812}}
 {'bbox': array([384.71234131, 187.26092529,   7.74884033,  25.22695923]), 'category_id': 39, 'category_name': 'bottle', 'id': 'dc2e6c8f-84a1-470e-9251-a67ebbe41175', 'label_id': '615c2cfa-7029-4bcf-b6d0-9a5d33a88c6b', 'metadata': {'iou': 0.642760157585144, 'score': 0.8809579014778137}}
 {'bbox': array([135.70680237, 193.470382

IndexError: list index out of range